# 01_summary_statistics.ipynb

In [1]:
import contextily as cx
import figure_utilities
import figure_and_table_constants
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
plt.rcParams["figure.dpi"] = 300
plt.rcParams['savefig.dpi'] = 300
import numpy as np
import os
import pandas as pd
from scipy import stats
import xyzservices.providers as xyz

# Store paths.
INPUT_DATA_UNRESTRICTED = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/data/03_cleaned/unrestricted.csv"
INPUT_DATA_RESTRICTED = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/data/03_cleaned/restricted.csv"
OUTPUT_TABLES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/summary_statistics/tables"
OUTPUT_FIGURES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/summary_statistics/figures"

# Read unrestricted dataset into memory.
unrestricted_df = pd.read_csv(INPUT_DATA_UNRESTRICTED)

# Drop malformed addresses.
unrestricted_df = unrestricted_df.loc[~unrestricted_df['property_address_full'].str.contains("span, span span"), :]

/Users/arjunshanmugam/Documents/GitHub/seniorthesis/venv/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Plot evictions spatially.
unrestricted_gdf = gpd.GeoDataFrame(unrestricted_df,
                                    geometry=gpd.points_from_xy(unrestricted_df['longitude'],
                                                                unrestricted_df['latitude']))
unrestricted_gdf = unrestricted_gdf.set_crs("EPSG:4326")
unrestricted_gdf = unrestricted_gdf.to_crs("EPSG:3857")
fig, ax = plt.subplots(figsize=(11, 12))
ax.set_yticklabels([]) 
ax.set_yticks([])
ax.set_xticklabels([]) 
ax.set_xticks([])
unrestricted_gdf.plot(ax=ax, color=figure_and_table_constants.Colors.P3, markersize=0.05)
cx.add_basemap(ax=ax, crs="EPSG:3857", source=cx.providers.Stamen.TonerLite)
figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "evictions_map.png"))

In [3]:
# Master summary statistics function.
def produce_summary_statistics(df: pd.DataFrame):
    # Panel A: Case Initiaton
    panel_A_columns = ['for_cause', 'foreclosure', 'no_cause', 'non_payment', 'for_cause_transfer', 'foreclosure_transfer',
                       'non_payment_transfer', 'no_cause_transfer']
    origin_columns = ['initiating_action', 'initiating_action', 'initiating_action','initiating_action', 
                      'initiating_action', 'initiating_action', 'initiating_action', 'initiating_action']
    target_values = ["SP Summons and Complaint - Cause",  "SP Summons and Complaint - Foreclosure",
                     "SP Summons and Complaint - No Cause", "SP Summons and Complaint - Non-payment of Rent",
                     "SP Transfer - Cause", "SP Transfer - Foreclosure", "SP Transfer - Non-payment of Rent",
                     "SP Transfer- No Cause"]

    for dummy_column, origin_column, target_value in zip(panel_A_columns, origin_columns, target_values):
        df.loc[:, dummy_column] = np.where(df[origin_column] == target_value, 1, 0)

    panel_A = df[sorted(panel_A_columns)].describe().T
    panel_A = pd.concat([panel_A], keys=["Panel A: Case Initiation"])

    # Panel B: Case Resolution
    panel_B_columns = ['mediated', 'dismissed', 'defaulted', 'heard']
    origin_columns = ['disposition_found', 'disposition_found', 'disposition_found',
                      'disposition_found']
    target_values = ["Mediated", "Dismissed", "Defaulted", "Heard"]

    for dummy_column, origin_column, target_value in zip(panel_B_columns, origin_columns, target_values):
        df.loc[:, dummy_column] = np.where(df[origin_column] == target_value, 1, 0)

    panel_B = df[sorted(panel_B_columns)].describe().T
    panel_B = pd.concat([panel_B], keys=["Panel B: Case Resolution"])

    # Panel C: Defendant and Plaintiff Characteristics
    panel_C_columns = ['hasAttyD', 'isEntityD', 'hasAttyP', 'isEntityP', 'judgment']
    panel_C = df[sorted(panel_C_columns)].describe().T
    panel_C = pd.concat([panel_C], keys=["Panel C: Defendant and Plaintiff Characteristics"])

    # Panel D: Tax Assessment Records From F.Y. Following Eviction Filing
    panel_D_columns = ['TOTAL_VAL', 'BLDG_VAL', 'LAND_VAL', 'OTHER_VAL', 'UNITS']
    panel_D = df[sorted(panel_D_columns)].describe().T
    panel_D = pd.concat([panel_D], keys=["Panel C: Assessor Records From Post-Filing F.Y."])

    # Panel E: Zestimates Around Treatment Time
    # Get month of latest docket date for each row and use to grab Zestimates at different times prior to treatment.
    df.loc[:, 'latest_docket_date'] = pd.to_datetime(df['latest_docket_date'])
    df.loc[:, 'nan'] = np.nan
    panel_E_columns = []
    for i in range(-5, 4):
        # This column contains the year-month which is i years relative to treatment for each property.
        offset_docket_month = (df['latest_docket_date'] + pd.tseries.offsets.DateOffset(years=i)).dt.strftime('%Y-%m').copy()

        # Some of the year-months will be outside of the range of our data.
        # For instance, we do not have Zestimates 2 years post-treatment for evictions which occurred in 2022. 
        # For these observations, the offset docket month needs to map to the column of nans we created earlier.
        offset_docket_month.loc[~offset_docket_month.isin(df.columns)] = 'nan'

        # Set column accordingly. 
        idx, cols = pd.factorize(offset_docket_month)
        new_col_name = f'zestimate_{i}_years_relative_to_treatment'
        panel_E_columns.append(new_col_name)
        df.loc[:, new_col_name] = df.reindex(cols, axis=1).to_numpy()[np.arange(len(df)), idx]

    panel_E = df[panel_E_columns].describe().T
    panel_E = pd.concat([panel_E], keys=["Panel E: Zestimates Around Treatment"])


    # Concatenate Panels A-E
    summary_statistics = pd.concat([panel_A, panel_B, panel_C, panel_D, panel_E], axis=0)[['mean', 'std', 'count']]

    return summary_statistics

In [4]:
# Produce summary statistics for unrestricted sample.
summary_statistics_unrestricted = produce_summary_statistics(unrestricted_df)

# Rename rows.
variable_names = summary_statistics_unrestricted.index.get_level_values(1)
display_names = ["For cause", "For cause (transfer)",
                "Foreclosure", "Foreclosure (transfer)",
                 "No cause", "No cause (transfer)",
                 "Non-payment of rent", "Non-payment of rent (transfer)",
                 "Case defaulted", "Case involuntarily dismissed", "Case heard", "Case mediated",
                 "Case voluntarily dismissed", "Defendant has an attorney", "Plaintiff has an attorney",
                "Defendant is an entity", "Plaintiff is an entity", "Money judgment", "Total property value",
                "Building value", "Land value", "Other value", "Units",
                "Five years before latest docket date", "Four years before latest docket date",
                 "Three years before latest docket date", "Two years before latest docket date", 
                "One year before latest docket date", "Latest docket date", "One year after latest docket date",
                "Two years after latest docket date", "Three years after latest docket date"] 
variable_display_names_dict = {key: value for key, value in zip(variable_names, display_names)}

summary_statistics_unrestricted = summary_statistics_unrestricted.rename(index=variable_display_names_dict)


# Rename columns.
summary_statistics_unrestricted.index = summary_statistics_unrestricted.index.set_names(["Panel", "Variable"])
column_display_names_dict = {'mean': "Mean", 'std': "S.D.", 'count': "N"}
summary_statistics_unrestricted = summary_statistics_unrestricted.rename(columns=column_display_names_dict)

# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "summary_statistics.tex")
latex = (summary_statistics_unrestricted
         .style
         .format(thousands=",",
                 formatter={'Mean': "{:.2f}",
                            'S.D.': "{:.2f}",
                            'N': "{:,.0f}"})
         .format_index("\\textit{{{}}}", escape="latex", axis=0, level=0)
         .to_latex(None,
                   column_format="llccc",
                   hrules=True,
                   clines="skip-last;data")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)

In [7]:
# Load restricted data.
restricted_df = pd.read_csv(INPUT_DATA_RESTRICTED)

# Separate into treatment and control groups.
restricted_df_treatment = restricted_df.loc[restricted_df['judgment_for_plaintiff'] == 1, :].copy()
restricted_df_control = restricted_df.loc[restricted_df['judgment_for_plaintiff'] == 0, :].copy()

# Produce summary statistics for treatment group.
summary_statistics_treatment = produce_summary_statistics(restricted_df_treatment).drop(columns='std')
summary_statistics_treatment = pd.concat({"Cases Won by Plaintiff": summary_statistics_treatment}, axis=1, names=["", ""])

# Produce summary statistics for control group. 
summary_statistics_control = produce_summary_statistics(restricted_df_control).drop(columns='std')
summary_statistics_control = pd.concat({"Cases Won by Defendant": summary_statistics_control}, axis=1, names=["", ""])

# Combine tables.
summary_statistics_restricted = pd.concat([summary_statistics_control, summary_statistics_treatment], axis=1)

# Rename rows.
summary_statistics_restricted = summary_statistics_restricted.rename(index=variable_display_names_dict)

# Rename columns.
summary_statistics_restricted.index = summary_statistics_restricted.index.set_names(["Panel", "Variable"])
column_display_names_dict = {'mean': "Mean", 'std': "S.D.", 'count': "N"}
summary_statistics_restricted = summary_statistics_restricted.rename(columns=column_display_names_dict)

# Export to LaTeX.
filename = os.path.join(OUTPUT_TABLES, "balance_table.tex")
latex = (summary_statistics_restricted
 .style
 .format(thousands=",", formatter={('Cases Won by Defendant', 'Mean'): "{:.2f}",

                                                      ('Cases Won by Defendant', 'N'): "{:.0f}",
                                                      ('Cases Won by Plaintiff', 'Mean'): "{:.2f}",

                                                      ('Cases Won by Plaintiff', 'N'): "{:.0f}" })
 .format_index("\\textit{{{}}}", escape="latex", axis=0, level=0)
 .format_index("\\textit{{{}}}", escape="latex", axis=1, level=0)
 .to_latex(None,
           column_format="llcccc",
           hrules=True,
           clines="skip-last;data",
           multicol_align="c")).replace("{*}", "{4cm}")
with open(filename, 'w') as file:
    file.write(latex)

In [8]:
# Plot distribution of the outcome variable in the treated group and the control group. 

# Store a list of column names containing Zestimates.
years = [str(year) for year in range(2013, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = ["2012-12"] + [str(year) + "-" + str(month) for year in years for month in months]

# Drop rows which could not be matched to any Zestimates.
has_all_zestimates = restricted_df[value_vars].notna().all(axis=1) 
print(f"Limiting sample to {has_all_zestimates.sum()} evictions for which we observe all Zestimates.")
restricted_df = restricted_df.loc[has_all_zestimates, :]


# Create dictionaries to map between string represetation and integer representation of each month.
month_to_int_dictionary = {key: value for value, key in enumerate(value_vars)}
int_to_month_dictionary = {key: value for key, value in enumerate(value_vars)}

# Reshape from wide to long.
restricted_df = pd.melt(restricted_df,
             id_vars=['case_number', 'latest_docket_date', 'judgment_for_plaintiff'],
             value_vars=value_vars, var_name='month', value_name='zestimate')
restricted_df = restricted_df.sort_values(by=['case_number', 'month'])

# Get 'first treated' date for each observation.
restricted_df.loc[:, 'latest_docket_date'] = pd.to_datetime(restricted_df['latest_docket_date']).dt.strftime('%Y-%m')

# Convert months from string format to integer format.
restricted_df.loc[:, 'month'] = restricted_df['month'].replace(month_to_int_dictionary)
restricted_df.loc[:, 'latest_docket_date'] = restricted_df['latest_docket_date'].replace(month_to_int_dictionary)

# Generate month relative to treatment.
restricted_df.loc[:, 'month_relative_to_treatment'] = ((restricted_df['month'] - restricted_df['latest_docket_date'])
                                                       .astype(int))

# Keep only observations observed at every treatment-relative time period from t=-10 to t=24.
around_treatment_mask = ((-10 <= restricted_df['month_relative_to_treatment']) &
                         (restricted_df['month_relative_to_treatment'] <= 24))
print(f"Limiting the sample to {around_treatment_mask.sum()} observations from treatment-relative months -10 "
      "to 24. Note that many of these observations are missing as not all properties have observed Zestimates"
      " in all time periods t=-10 to t=24.")
restricted_df = restricted_df.loc[around_treatment_mask, :] 

# Keep only rows corresponding to cases which are observed from t=-10 to t=24.
def check_if_time_periods_observed(s: pd.Series, start: int, end: int):
    values = pd.Series(range(start, end + 1))
    check = values.isin(s)
    return check.all()
valid_case_numbers = (restricted_df
               .groupby('case_number')['month_relative_to_treatment']
               .apply(lambda x: check_if_time_periods_observed(x, -10, 24)))
valid_case_numbers = valid_case_numbers[valid_case_numbers].index
valid_case_numbers_mask = restricted_df['case_number'].isin(valid_case_numbers)
print(f"Limiting the sample to {valid_case_numbers_mask.sum()} observations which correspond to cases observed "
       "for every month from treatment-relative month -10 to treatment-relative month 24.")
restricted_df = restricted_df.loc[valid_case_numbers_mask, :]

# Aggregate by treatment-relative month and treatment status.
restricted_df = (restricted_df.groupby(['month_relative_to_treatment', 'judgment_for_plaintiff'])['zestimate']
                 .agg([np.mean, np.std, 'count'])
                )
restricted_df.loc[:, 'se'] = restricted_df['std'] / np.sqrt(restricted_df['count']) 
restricted_df.loc[:, 'y_upper'] = restricted_df['mean'] + 1.96*restricted_df['se']
restricted_df.loc[:, 'y_lower'] = restricted_df['mean'] - 1.96*restricted_df['se']

# Separate into control and treatment group
treated_units = restricted_df.loc[(slice(None), 1), :].reset_index()
control_units = restricted_df.loc[(slice(None), 0), :].reset_index()

# Plot event study-style plot of trends in Zestimates.
fig, ax = plt.subplots()
ax.set_title("Trends in Zestimates")
ax.set_xlabel("Month Relative to Latest Docket Date")
ax.set_ylabel("Zestimate")
for dataset, color, marker, label in zip([treated_units, control_units],
                                         ['red', 'blue'],
                                         ["^", "s"],
                                         ["Cases Won By Plaintiff",
                                          "Cases Won By Defendant"]):     
    x = dataset['month_relative_to_treatment'].values
    y = dataset['mean'].values
    y_upper = dataset['y_upper'].values
    y_lower = dataset['y_lower'].values
    figure_utilities.plot_scatter_with_shaded_errors(ax=ax,
                                                      x=x,
                                                    y=y,
                                                     y_upper=y_upper,
                                                     y_lower=y_lower,
                                                     label=label,
                                                     point_color=color,
                                                     marker=marker,
                                                     error_color=color,
                                                    error_opacity=0.3)
ax.legend()    
figure_utilities.save_figure_and_close(fig, os.path.join(OUTPUT_FIGURES, "trends_in_zestimates.png"))
plt.show()



Limiting sample to 2657 evictions for which we observe all Zestimates.
Limiting the sample to 122303 observations from treatment-relative months -24 to 24. Note that many of these observations are missing as not all properties have observed Zestimates in all time periods t=-24 to t=24.
Limiting the sample to 75460 observations which correspond to cases observed for every month from treatment-relative month -24 to treatment-relative month 24.
